<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Création-du-compte-de-bot" data-toc-modified-id="Création-du-compte-de-bot-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Création du compte de bot</a></span></li><li><span><a href="#Utilisation-de-pywikiapi" data-toc-modified-id="Utilisation-de-pywikiapi-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Utilisation de pywikiapi</a></span><ul class="toc-item"><li><span><a href="#Login" data-toc-modified-id="Login-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Login</a></span></li><li><span><a href="#Lire-le-contenu-depuis-l'API" data-toc-modified-id="Lire-le-contenu-depuis-l'API-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Lire le contenu depuis l'API</a></span></li><li><span><a href="#Éditer-wikipast-en-python" data-toc-modified-id="Éditer-wikipast-en-python-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Éditer wikipast en python</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Conclusion</a></span></li></ul></li><li><span><a href="#Quelques-ressources-utiles-pour-Wikipast" data-toc-modified-id="Quelques-ressources-utiles-pour-Wikipast-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Quelques ressources utiles pour Wikipast</a></span></li></ul></div>

# Introduction

Comme nous le verrons plus tard dans le cours, Wikipedia est peuplé par de nombreux bots qui sont les utilisateurs les plus prolifiques en terme de modifications brutes. Wikimedia, le logiciel qui fait tourner Wikipedia, possède donc une API très complète permettant de réaliser facilement la plupart des tâches basiques, à savoir retrouver des pages et les modifier.

La documentation de cette API est trouvable en ligne à cette adresse: https://www.mediawiki.org/wiki/API:Main_page/fr. Cette API est utilisable à l'aide de requêtes web classiques, toutefois il existe plusieurs "wrappers" python permettant de l'utiliser plus facilement dans ce langage. Nous utiliserons dans ce tutoriel le wrapper [`pywikiapi`](https://github.com/nyurik/pywikiapi) qui est très léger et certainement suffisant pour notre cours, toutefois vous pouvez également jeter un coup d'oeil aux autres wrappers: https://www.mediawiki.org/wiki/API:Client_code.


# Création du compte de bot
Pour faire des appels de lectures à l'API, il n'est pas nécessaire d'avoir de compte, toutefois pour pouvoir modifier les pages, il est nécessaire de créer un compte de bot. Les étapes de créations sont les suivantes:

1. se rendre sur la page http://wikipast.epfl.ch/wikipast/index.php/Sp%C3%A9cial:BotPasswords/,
2. Créer un nouveau robot en choisissant les bons droits de modifications en fonction de vos besoins (par exemple: "Modification de gros volumes", "Modifier des pages existantes", "Créer, modifier et déplacer des pages", "Importer de nouveaux fichiers", "Téléverser, remplacer et renommer des fichiers")
3. La page suivante donne le mot de passe du bot, il faut bien le sauvegarder car il ne sera plus affiché, de plus il faut le garder secrer car il permet de modifier des pages sur Wikipast.


Vous pouvez maintenant rentrer les identifiants du bots dans la cellule ci-dessous pour pouvoir l'utiliser dans ce tutoriel (et ne pas oublier de l'exécuter).

In [ ]:
user = 'Username@yourbot'
password = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

# Utilisation de pywikiapi

## Login

In [ ]:
from pywikiapi import Site

`pywikiapi` utilise un objet `Site` pour faire ses requête. Cet objet prend l'adresse de l'API et permet d'abstraire la plupart des appels à l'API derrière une syntaxe python simple. Seules deux fonctionalités de l'API sont implémentées spécifiquement dans `pywikiapi`: `login` et `query`, toutefois, tous les autres fonctionnalités de l'API peuvent être également invoquées.

In [ ]:
site = Site('http://wikipast.epfl.ch/wikipast/api.php') # Définition de l'adresse de l'API
site.no_ssl = True # Désactivation du https, car pas activé sur wikipast
site.login(user, password) # Login du bot

Nous avons maintenant appelé la fonction [`login`](http://wikipast.epfl.ch/wikipast/api.php?action=help&modules=login) de l'API.

## Lire le contenu depuis l'API
Les deux autres fonctions implémentées qui appellent explicitement l'API sont `site.query` et `site.query_pages`. En voici un exemple:

In [ ]:
results = []
# Cherche dans toutes les pages celles commençant par Leopold
for res in site.query(list='allpages', apprefix='Mahatma'): 
    results.append(res)
results

Nous pouvons extraire de cette liste de résultats le nom des pages:

In [ ]:
pages_names = []
for res in results[0]['allpages']: # aller dans le dictionnaire nesté
    pages_names.append(res['title'])
pages_names

Une fois ces noms extraits, nous pouvons utiliser `site.query_pages` pour extraire certaines informations sur ces pages. La cellule suivante trouves les pages ayant un titre dans page_names et retourne les attributs "categories", "links", "extlinks" comme décrits dans la documentation: http://wikipast.epfl.ch/wikipast/api.php?action=help&modules=query.

In [ ]:
results_pages = []

for res in site.query_pages(titles=pages_names, prop=['categories', 'links', 'extlinks']): 
    results_pages.append(res)
results_pages

Toutefois, ce qui va souvent nous intéresser est le contenu textuelle d'une page du wiki. `pywikiapi` n'implémente pas directement des fonctions pour faire cela, mais nous pouvons tout de même l'utiliser grâce à un appel direct à `Site`.

En effet, en écrivant `site('parse', page='Mahatma Gandhi', prop=['wikitext'])`, `pywikiapi` fait directement un appel à la fonction `parse` de l'API, nous pouvons ensuite passer les autres arguments directement dans l'appel de la fonction et ils seront transmis à l'API. Pour plus de détails sur les arguments possibles, il faut de nouveau se référer à la documentation: http://wikipast.epfl.ch/wikipast/api.php?action=help&modules=parse. Dans celle-ci, nous pouvons voir que `parse` prend un argument `page` qui indique la page à parser et un argument `prop` qui peut entre autre prendre la valeur `sections` qui fait retourner à `parse` une liste de ses sections. En voici le résultat:

In [ ]:
res = site('parse', page='Mahatma Gandhi', prop=['sections'])
res

Maintenant que nous avons les indexs des sections, nous pouvons récupérer le wikitext d'une section particulière:

In [ ]:
res = site('parse', page='Mahatma Gandhi', prop=['wikitext'], section=None)
res

Nous pouvons maintenant écrire une fonction qui prend un nom de page et nous retourne son wikitext et une autre qui nous retourne ses sections:

In [ ]:
def get_wiki_text(page, section=None):
    result = site('parse', page=page, prop=['wikitext'], section=section)
    return result['parse']['wikitext']

def get_sections(page):
    result = site('parse', page=page, prop=['sections'])
    return result['parse']['sections']

Vous pouvez maintenant tester les fonctions:

In [ ]:
get_sections('Mahatma Gandhi')

In [ ]:
print(get_wiki_text('Mahatma Gandhi', section=1))

Voici un exemple de comment parser les entrées de datafication depuis le wikitext:

In [ ]:
sections = get_sections('Mahatma Gandhi')
section_biographie_id = None
for section in sections:
    if section['line'] == 'Biographie':
        section_biographie_id = section['index']
wikitext = get_wiki_text("Mahatma Gandhi", section=section_biographie_id)
entries = [entry for entry in wikitext.split("\n") if entry.startswith('*')]
entries


## Éditer wikipast en python

Il nous reste donc à montrer comment éditer une page, l'API est défini sur cette page: http://wikipast.epfl.ch/wikipast/api.php?action=help&modules=edit.

Vous pouvez librement vous créer une page bac à sable où utiliser la page [bacasable](http://wikipast.epfl.ch/wikipast/index.php/Bacasable).

Nous allons donc créer une nouvelle page et l'éditer depuis python, vous pouvez à chaque étape voir le résultat de vos modifications sur la page correspondante de wikipast.

Commençons par créer la nouvelle page (modifiez la variable `titre`), soyez prudents, ce bout de code écrase entièrement la page (même s'il existait déjà):

In [ ]:
titre = 'Bacasable - tutoriel'

site('edit', title=titre,
     text='Ceci est une nouvelle page.\nAvec peu de contenu.',
     token=site.token())

Il est important de noter que pour modifier une page (création, modification, suppression), il faut absolument être connecté et avoir un token d'édition. Ce dernier peut être obtenu en appelant `site.token()` et devra donc être ajouté comme argument à chaque fonction éditant une page.

Rajoutons maintenant deux nouvelles sections à notre page:

In [ ]:
site('edit', title=titre,
     section='new',
     sectiontitle='Nouvelle section 1 de test',
     text='Ceci est le texte de ma nouvelle section.',
     token=site.token())
site('edit', title=titre,
     section='new',
     sectiontitle='Nouvelle section 2 de test',
     text='Ceci est le texte de ma nouvelle section.',
     token=site.token())

Nous pouvons maintenant ajouter du texte à une de nos sections grâce aux arguments `prependtext` et `appendtext`:

In [ ]:
site('edit', title=titre,
     section=2,
     prependtext='Rajoutons du texte avant.\n', # notez l'ajout d'un retour à la fin du texte ajouté
     token=site.token())

site('edit', title=titre,
     section=2,
     appendtext='\nRajoutons du texte et après.\n',
     token=site.token())

Ou écraser le texte, mais d'une seule section.

In [ ]:
site('edit', title=titre,
     section=2,
     text='Nouveau texte', # notez l'ajout d'un retour à la fin du texte ajouté
     token=site.token())

Le seul moyen d'être plus fin dans la modification de la page est de récupérer le wikitext de la page, de modifier sa string, puis d'écraser la page avec le texe:

In [ ]:
text = get_wiki_text(titre)
text

In [ ]:
text = text.replace('Nouvelle section', 'Sous-titre')

In [ ]:
site('edit', title=titre,
    text=text,
    token=site.token())

## Conclusion

Nous avons donc vu comment récupérer des informations des pages de wikipast et comment éditer les pages. Toutefois, beaucoup d'options de l'API n'ont pas été couvertes ici, il est donc judicieux de s'y référer lorsque vous créerez vos bots afin de ne pas manquer une fonctionnalité qui pourrait déjà être implémentée avant de la réinventer vous-même.

# Quelques ressources utiles pour Wikipast

Afin de pouvoir traiter les ~670'000 pages présentes sur Wikipast, quelques optimisations sont nécessaires.

Voici donc quelques bouts de codes permettant de parcourir toutes les pages efficacement et deux fichiers json pré-calculés qui vous permettront de ne pas avoir à la refaire trop souvent. 

Les fichiers `json.gz` sont disponibles sur le github du tutoriel dans l'onglet [releases](https://github.com/dhlab-epfl/HUM-365-tutorials/releases).


In [ ]:
from tqdm.notebook import tqdm # Libraire utile pour voir le progrès d'une boucle
import multiprocessing as mp # Librairie d'execution parallèle
import gzip # Permet d'économiser beaucoup de place en compressant les fichiers json
import json

# Read a gzipped json file
def load_gzip_json(path):
    with gzip.GzipFile(path, 'r') as infile:
        return json.loads(infile.read().decode('utf-8'))

def write_gzip_json(data, path):
    with gzip.GzipFile(path, 'w') as outfile:
        outfile.write(json.dumps(data, ensure_ascii=False).encode('utf-8'))

Ce premier bout de code permet de récupérer les titres et id de toutes les pages du wiki, il prend ~1h à être exécuté. La version actuelle a été compilée le 15.03.

In [ ]:
# all_pages = []
# for r in tqdm(site.query(list='allpages')):
#     for page in r['allpages']:
#         all_pages.append(page)
# write_gzip_json(all_pages, 'all_pages.json.gz')

In [ ]:
all_pages = load_gzip_json('./all_pages.json.gz')
all_pages[100010:100020]

Ce deuxième bout de code récupère la plupart des infos intéressante que l'on pourrait vouloir d'une page. Il lance le processus en parralèle sur 16 threads, rendant le tout beaucoup plus rapide. Le pattern de code de parallèlisation peut-être réutiliser pour vos propre bots qui devraient traiter beaucoup de pages.

Attention, ce code qui traite autant de page est lourd pour nos serveurs et est long à éxecuter (~2h30). La version actuelle a été compilée le 15.03.

In [ ]:
# def get_all_page_infos(page):
#     return site('parse', page=page, prop=['wikitext','links',
#                                           'externallinks', 'categories',
#                                           'revid', 'images', 'sections'])

# pages_titles = [page['title'] for page in all_pages] # la syntaxe est une list comprehension, c.f. https://docs.python.org/fr/3/tutorial/datastructures.html#list-comprehensions

# # Création d'une pool avec 16 process
# pool = mp.Pool(16)
# # Création d'une barre de chargement et d'une fonction pour la mettre à jour.
# pbar = tqdm(total=len(pages_titles))
# def update(*a):
#     pbar.update()

# # Récupération de tous les résultats en parallèle
# results = []
# for page in pages_titles:
#     results.append(pool.apply_async(get_all_page_infos, args=(page,), callback=update))
# pool.close()
# pool.join()

# # Certaines requêtes ont inévitablement raté, il faut donc récupérer le reste
# all_results = []
# num_failures = 0
# for idx, result in enumerate(results):
#     try:
#         all_results.append(result.get()['parse'])
#     except:
#         num_failures += 1
# print(f"There is {num_failures} that failed.")

# # Trouvons maintenant les titre des pages qui ont été récupérées
# valid_pages_titles = set([result['title'] for result in all_results])
    
# # Trouvons maintenant les pages qui n'ont pas été récupérées
# missing_pages_titles = set(pages_titles).difference(valid_pages_titles)

# # Et récuperons les
# for page in missing_pages_titles:
#     all_results.append(get_all_page_infos(page))
    
# print(f"We now have {len(all_results)} out of {len(all_pages)}.")


# # Finalement, écrivons le fichier avec les résultats
# write_gzip_json(all_results, 'all_data.json.gz')

In [ ]:
all_data = load_gzip_json('./all_data.json.gz')
all_data[100000:100002]

Ces deux fichiers json peuvent donc être utiles pour explorer les données sans avoir à faire des querys sur toute la base de données.

Finalement, écrivons deux query qui peuvent être utiles comme points de départ pour un bot. 

La première nous permet de récuper tous les liens présents sur la page de biographies et qui peut être utile comme point de départ.

In [ ]:
site('parse', page='Biographies', prop='links')['parse']['links']

La deuxième utilise le fait que certains bots ont ajouté un lien externe vers l'attribut [Q5](https://www.wikidata.org/wiki/Q5) de wikidata qui indique que la page parle d'un humain, cela peut donc être utilisé pour récupérer toutes les pages parlants d'humains.

In [ ]:
# Vérifie si https://www.wikidata.org/wiki/Q5 est dans la liste de liens externes
# Utilise une liste comprehension avec un filtre conditionnel
humans = [data['title'] for data in all_data if 
          'externallinks' in data and
          'https://www.wikidata.org/wiki/Q5' in data['externallinks']]

In [ ]:
print(f"There is {len(humans)} pages about humans.")
humans[10000:10010]

Une dernière note, si vous faites des modifications massives. Soyez prudents en vérifiant avant chaque modification que le `revid` de la page que vous avez récuperé avant modification est le même que celui de la page qui va être modifiée. C'est-à-dire, si vous précalculez toutes les pages à être modifiées, récuperez également leurs `revid`. Ensuite, au moment de la modification, vérifier que le `revid` de la page qui est en ligne est le même que le vôtre, sinon vous allez écrasé une nouvelle modification.